In [1]:
# Import netket library
import netket as nk
from qiskit.chemistry import FermionicOperator
import itertools

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

/home/nico/anaconda3/envs/scqs_env/lib/python3.9/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [10]:
import netket as nk
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry import FermionicOperator

def JW_H(systemData={'driver_string': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54', 'basis': 'sto3g'}):
                            
    driver = PySCFDriver(   atom=systemData["driver_string"],
                            basis=systemData["basis"]       )
                            
    mol = driver.run()
    OB = mol.one_body_integrals
    TB = mol.two_body_integrals

    FerOp = FermionicOperator(OB, TB)
    mapping = FerOp.mapping('jordan_wigner')
    weights = [w[0] for w in mapping.paulis]
    operators = [w[1].to_label() for w in mapping.paulis]

    return nk.operator.PauliStrings(operators, weights), mol.nuclear_repulsion_energy

In [2]:
import netket as nk
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry import FermionicOperator

systemData={'driver_string': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54', 'basis': 'sto3g'}
systemData={'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': 'sto3g'}

driver = PySCFDriver(   atom=systemData["driver_string"],
                        basis=systemData["basis"]       )
                        
mol = driver.run()
OB = mol.one_body_integrals
TB = mol.two_body_integrals

FerOp = FermionicOperator(OB, TB)
mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]
ha = nk.operator.PauliStrings(operators, weights)

/home/nico/anaconda3/envs/scqs_env/lib/python3.9/site-packages/qiskit/chemistry/fermionic_operator.py:386: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  pauli_list = WeightedPauliOperator(paulis=[])
/home/nico/anaconda3/envs/scqs_env/lib/python3.9/site-packages/qiskit/chemistry/fermionic_operator.py:394: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.aqua_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  task_args=(threshold,), num_processes=aqua_globals.num_processes)


In [3]:
#print(weights)
print(operators)
print("Only unique operators: ", len(operators)==len(np.unique(operators)))

['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'XXYY', 'YYYY', 'XXXX', 'YYXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII']
Only unique operators:  True


In [4]:
print(ha.hilbert.state_to_number(np.array([0,0,1,1])))
print(ha.hilbert.state_to_number(np.array([1,1,0,0])))

AttributeError: 'Qubit' object has no attribute 'state_to_number'

In [6]:
ha.get_conn(np.array([0,0,1,1]))

(array([[0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.]]),
 array([-1.24432134+0.j,  0.04521886+0.j, -0.04521886+0.j]))

In [7]:
print(ha.to_sparse())

  (0, 15)	(0.04521886247455256+0j)
  (0, 15)	(-0.04521886247455256+0j)
  (1, 1)	(-1.256946259950978+0j)
  (1, 14)	(0.04521886247455256+0j)
  (1, 14)	(-0.04521886247455256+0j)
  (2, 2)	(-0.4712556865101022+0j)
  (2, 13)	(0.04521886247455256+0j)
  (2, 13)	(-0.04521886247455256+0j)
  (3, 3)	(-1.2443213422905408+0j)
  (3, 12)	(0.04521886247455256+0j)
  (3, 12)	(-0.04521886247455256+0j)
  (4, 4)	(-1.2569462599509775+0j)
  (4, 11)	(-0.04521886247455256+0j)
  (4, 11)	(0.04521886247455256+0j)
  (5, 5)	(-1.8379915067757564+0j)
  (5, 10)	(0.1356565874236577+0j)
  (5, 10)	(0.04521886247455256+0j)
  (6, 6)	(-1.06344589239233+0j)
  (6, 9)	(0.1356565874236577+0j)
  (6, 9)	(0.04521886247455256+0j)
  (7, 7)	(-1.1606105350465692+0j)
  (7, 8)	(-0.04521886247455256+0j)
  (7, 8)	(0.04521886247455256+0j)
  (8, 8)	(-0.4712556865101019+0j)
  (8, 7)	(-0.04521886247455256+0j)
  (8, 7)	(0.04521886247455256+0j)
  (9, 9)	(-1.0634458923923302+0j)
  (9, 6)	(0.1356565874236577+0j)
  (9, 6)	(0.04521886247455256+0j)
 

In [3]:
ha, nre = JW_H({'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': 'sto3g'})
hi = ha.hilbert
n_electrons=2

/Users/yapple/.pyenv/versions/miniconda3-4.7.12/envs/nk3/lib/python3.8/site-packages/qiskit/chemistry/fermionic_operator.py:386: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  pauli_list = WeightedPauliOperator(paulis=[])
/Users/yapple/.pyenv/versions/miniconda3-4.7.12/envs/nk3/lib/python3.8/site-packages/qiskit/chemistry/fermionic_operator.py:394: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.aqua_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  task_args=(threshold,), num_processes=aqua_globals.num_processes)


In [8]:
ha, nre = JW_H({'driver_string': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54', 'basis': 'sto3g'})
hi = ha.hilbert
n_electrons=4

In [ ]:
ha, nre = JW_H({'driver_string': 'C 0.0 0.0 0.0; C 0.0 0.0 1.26', 'basis': 'sto3g'})
hi = ha.hilbert
n_electrons=12

In [4]:
res = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=False)
print("Exact ground state energy = {}".format(res+nre))

Exact ground state energy = [-1.13730541]


## EERSTE STUK NK3

In [ ]:
print(ha.is_hermitian)

In [ ]:
#nk.config.update("NETKET_EXPERIMENTAL", 1)

In [ ]:
ma = nk.models.RBM(alpha=2, use_visible_bias=True, use_hidden_bias=True, dtype=complex, kernel_init=nk.nn.initializers.normal(stddev=0.1))
g = nk.graph.Hypercube(length=hi.size, pbc=False)
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g)


vs = nk.variational.MCState(sa, ma, n_samples=20000)
vs.init_parameters(nk.nn.initializers.normal(stddev=0.1))

op = nk.optimizer.Adam(learning_rate=0.01)
sr = nk.optimizer.SR(diag_shift=0.1) # Default 0.01

vmc = nk.VMC(
        hamiltonian=ha,
        optimizer=op,
        sr=sr,
        variational_state=vs)

In [ ]:
for it in vmc.iter(500,5):
    print(it,vmc.energy)

## VANAF HIER NK2.9

In [5]:
from netket.machine.jax import JaxRbm
ma = JaxRbm(hi, alpha=2, dtype=complex)
chain_length=16

ma.init_random_parameters(seed=1234, sigma=0.05)
sa = nk.sampler.MetropolisLocal(machine=ma, n_chains=chain_length)

In [ ]:
g = nk.graph.Hypercube(length=hi.size, pbc=False)
sa = nk.sampler.MetropolisExchange(machine=ma, graph=g, n_chains=chain_length)

n_up = []
tries = 20000
for i in range(tries):
    n_up = []
    sa = nk.sampler.MetropolisExchange(machine=ma, graph=g, n_chains=chain_length)
    for ss in sa.samples(1):
        for s in ss:
             #print(s, list(s).count(1))
             n_up.append(int(list(s).count(1)))
    if n_up.count(n_electrons) == chain_length: print('found after %d tries' %(i)); break
if i+1==tries:
    print('not found')
else:
    for s in sa.samples(2):
        print(s)

In [7]:
I = []
E = []

In [8]:
# Optimizer
opt = nk.optimizer.Sgd(ma, learning_rate=0.1)
#opt = nk.optimizer.AdaMax(ma, alpha=0.1)

# Stochastic reconfiguration
sr = nk.optimizer.SR(ma, diag_shift=0.1, use_iterative=True)
    
# The ground-state optimization loop
vmc = nk.Vmc(
            hamiltonian=ha,
            sampler=sa,
            optimizer=opt,
            n_samples=100000,
            #n_discard=10*hi.size,
            sr=sr)

for it in vmc.iter(400,5):
    print(it,vmc.energy, vmc.energy.mean.real+nre)
    I.append(it)
    E.append(vmc.energy.mean.real)

/Users/yapple/.pyenv/versions/miniconda3-4.7.12/envs/nk3/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:2905: ComplexWarning: Casting complex values to real discards the imaginary part
  out = lax.convert_element_type(out, dtype, weak_type=weak_type)
0 -5.0279+0.0000j ± 0.0070 [σ²=3.4048, R̂=1.0000] -3.9970270510111274


In [ ]:
plt.plot(I,E)

In [ ]:
i=0
while (vmc._samples[-i]==vmc._samples[-1]).all():
    i+=1
print(vmc._samples[-1],vmc._samples[-i], i)

In [ ]:
g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
hi = nk.hilbert.Spin(s=0.5, N=g.n_nodes, total_sz=0) # Remove sector!!
ha = nk.operator.Heisenberg(graph=g, hilbert=hi)